In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
from keras import backend as K

print(tf.keras.__version__)
print(K.tensorflow_backend._get_available_gpus())

Using TensorFlow backend.


2.2.4-tf
['/job:localhost/replica:0/task:0/device:GPU:0']


In [2]:
### optional - Inspect Data Generator
from keras.preprocessing.image import array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.15,
        horizontal_flip=False,
        fill_mode='nearest')

img = load_img('./simple_clock/Datagenerator_simple_clock/Training/42/clock-04.07.00.png')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir='preview', save_prefix='augmentation', save_format='png'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

In [2]:
### model
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(56, 52, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
#model.add(Dropout(0.5))
model.add(Dense(60))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
### Data preperation

batch_size = 10

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        #rotation_range=5,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.15,
        horizontal_flip=False,
        fill_mode='nearest')

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data

#                    color_mode = 'grayscale',
#                    class_mode = 'sparse',
#                    shuffle = True,
#                    save_to_dir = '.\augmentation')
train_generator = train_datagen.flow_from_directory(
        'simple_clock/Datagenerator_simple_clock/Training',  # this is the target directory
        target_size=(56, 52),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',  # since we use binary_crossentropy loss, we need binary labels
        color_mode = 'rgb',
        shuffle = True,
        save_to_dir = 'simple_clock/augmentation')

# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'simple_clock/Datagenerator_simple_clock/Testing',
        target_size=(56, 52),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode = 'rgb',
        shuffle = True,
        save_to_dir = 'simple_clock/augmentation')

Found 660 images belonging to 60 classes.
Found 60 images belonging to 60 classes.


In [4]:
### Training
model.fit_generator(
        train_generator,
        steps_per_epoch=(720-60) // batch_size,
        epochs=50,
        #validation_data=validation_generator,
        #validation_steps=60 // batch_size
        )
model.save_weights('simple_clock_classification_first_try.h5')  # always save your weights after training or during training

Instructions for updating:
Use tf.cast instead.
Epoch 1/50


InternalError: CUB segmented reduce errorinvalid configuration argument
	 [[{{node training/Adam/gradients/conv2d_1/BiasAdd_grad/BiasAddGrad}}]]